## Deep Learning Classifier 

### Melinda Xiao-Devins

Implement deep learning neural network classifier


In [136]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
import os

In [137]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
#from tensorflow.python.layers.core import Dense
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))
# Check for a GPU
if not tf.test.gpu_device_name():
 warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
 print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


In [138]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [139]:
# load dataset
#df = pd.read_csv('./data/ny_hmda_2015_normalize.csv', low_memory=False, header=0, delimiter=",")
df = pd.read_csv('./data/ny_hmda_2015_normalize.csv', low_memory=False, header=0, delimiter=",")

#print(dataframe.loc[:,:])
num_rows = df.shape[0]
num_col = df.shape[1]
print ("Total number of records: {}".format(num_rows))
print ("Toatl numver of features: {}".format(num_col))

X = np.array(df.drop(['action_taken'],1)) 
Y = np.array(df['action_taken'])

#Split into train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Total number of records: 376516
Toatl numver of features: 38


In [140]:
# Create a fully connected network with 4 layers
model = Sequential()

#input layer, it has 38 neurons, it must have right number of inputs, which is the number of features
model.add(Dense(38, input_dim=num_col-1, kernel_initializer='normal', activation='relu'))

# hideen layer has 32 neurons
model.add(Dense(32, kernel_initializer='normal', activation='relu'))

# hideen layer has 16 neurons
model.add(Dense(16, kernel_initializer='normal', activation='relu'))

# hideen layer has 8 neurons
model.add(Dense(8, kernel_initializer='normal', activation='relu'))

# output layer has 1 neuron to predict
# Use sigmoid for output layer activation function to ensure network output is bw. 0 and 1
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [141]:
#Compile model

# loss function: logarithmic loss, which is binary_crossentropy for binary classification
# use 'adam' optimizer for gradient descent algorithm 
# collect accuracy during training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [142]:
# Train the model
# epochs: a fixed number of iterations through the dataset
# batch size: the number of instances that are evaluated before a weight update in the network is performed 
model.fit(X_train, Y_train, epochs = 500, batch_size=100)

Epoch 1/500
301212/301212 [==============================] - 16s 52us/step - loss: 0.3569 - acc: 0.8125
Epoch 2/500
301212/301212 [==============================] - 14s 48us/step - loss: 0.3407 - acc: 0.8243
Epoch 3/500
301212/301212 [==============================] - 14s 48us/step - loss: 0.3371 - acc: 0.8280
Epoch 4/500
301212/301212 [==============================] - 15s 50us/step - loss: 0.3354 - acc: 0.8293
Epoch 5/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3337 - acc: 0.8297
Epoch 6/500
301212/301212 [==============================] - 14s 48us/step - loss: 0.3325 - acc: 0.8314
Epoch 7/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3316 - acc: 0.8310
Epoch 8/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3301 - acc: 0.8321
Epoch 9/500
301212/301212 [==============================] - 14s 48us/step - loss: 0.3290 - acc: 0.8328
Epoch 10/500
301212/301212 [==============================] - 14

301212/301212 [==============================] - 13s 45us/step - loss: 0.3184 - acc: 0.8416
Epoch 156/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3183 - acc: 0.8411
Epoch 157/500
301212/301212 [==============================] - 13s 45us/step - loss: 0.3182 - acc: 0.8412
Epoch 158/500
301212/301212 [==============================] - 13s 45us/step - loss: 0.3184 - acc: 0.8412
Epoch 159/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3181 - acc: 0.8414
Epoch 160/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3181 - acc: 0.8417
Epoch 161/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3180 - acc: 0.8414
Epoch 162/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3181 - acc: 0.8414
Epoch 163/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3180 - acc: 0.8412
Epoch 164/500
301212/301212 [==============================]

301212/301212 [==============================] - 13s 45us/step - loss: 0.3152 - acc: 0.8427
Epoch 310/500
301212/301212 [==============================] - 13s 45us/step - loss: 0.3150 - acc: 0.8429
Epoch 311/500
301212/301212 [==============================] - 13s 45us/step - loss: 0.3152 - acc: 0.8431
Epoch 312/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3148 - acc: 0.8430
Epoch 313/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3151 - acc: 0.8434
Epoch 314/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3150 - acc: 0.8434
Epoch 315/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3150 - acc: 0.8433
Epoch 316/500
301212/301212 [==============================] - 13s 45us/step - loss: 0.3148 - acc: 0.8432
Epoch 317/500
301212/301212 [==============================] - 13s 44us/step - loss: 0.3151 - acc: 0.8431
Epoch 318/500
301212/301212 [==============================]

301212/301212 [==============================] - 14s 47us/step - loss: 0.3136 - acc: 0.8444
Epoch 464/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3136 - acc: 0.8439
Epoch 465/500
301212/301212 [==============================] - 14s 48us/step - loss: 0.3134 - acc: 0.8445
Epoch 466/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3135 - acc: 0.8441
Epoch 467/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3139 - acc: 0.8438
Epoch 468/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3135 - acc: 0.8441
Epoch 469/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3136 - acc: 0.8447
Epoch 470/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3134 - acc: 0.8443
Epoch 471/500
301212/301212 [==============================] - 14s 47us/step - loss: 0.3135 - acc: 0.8442
Epoch 472/500
301212/301212 [==============================]

In [143]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

75304/75304 [==============================] - 5s 62us/step

acc: 84.20%


In [144]:
# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])


the prediction bw. 0 and 1
[[ 0.57734799]
 [ 1.        ]
 [ 0.71808767]
 ..., 
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]
the rounded prediction
[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]


In [145]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(Y_test, [round(x[0]) for x in model.predict(X_test)],average="macro")
print("precision={}, recall={}, fscore={}, support={}".format(precision, recall, fscore, support))


precision=0.8191370203321797, recall=0.8200183019751683, fscore=0.8195743073445072, support=None


## Save Trained Model
Save the trained model to disk

In [146]:
 
# serialize model to JSON
model_json = model.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model/model.h5")
print("Saved model to disk")

Saved model to disk


## Load Saved Model
Load the saved model, and used it. It saves training time

In [147]:
# load json and create model
json_file = open('model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model/model.h5")
print("Loaded model from disk")
 
print("Use loaded model to predict")
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])



Loaded model from disk
Use loaded model to predict
the prediction bw. 0 and 1
[[ 0.57734799]
 [ 1.        ]
 [ 0.71808767]
 ..., 
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]
the rounded prediction
[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]


In [148]:
score = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("From saved model,  %s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))



From saved model,  acc: 84.48%


In [149]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(Y_test, [round(x[0]) for x in model.predict(X_test)],average="macro")
print("precision={}, recall={}, fscore={}, support={}".format(precision, recall, fscore, support))


precision=0.8191370203321797, recall=0.8200183019751683, fscore=0.8195743073445072, support=None
